In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clean_raw_data.csv
/kaggle/input/clean_data_mean.csv
/kaggle/input/raw_data.csv
/kaggle/input/clean_data_GCN.csv
/kaggle/input/FillZero_minmax_baseline/clean_data_minmax_fill-zero.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/val/val_week1_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/test/test_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/clean_data_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_4.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_1.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_5.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/val/val_week1_2_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/test/test_week3.csv
/kaggle/input/FillZero_minmax_bas

In [2]:
!pip uninstall -y scikit-learn imbalanced-learn

Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2
Found existing installation: imbalanced-learn 0.13.0
Uninstalling imbalanced-learn-0.13.0:
  Successfully uninstalled imbalanced-learn-0.13.0


In [3]:
!pip install scikit-learn==1.4.2 imbalanced-learn==0.12.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.0/258.0 kB 15.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.


In [4]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import StratifiedKFold
import time
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_auc_score

2025-06-30 10:34:55.481383: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751279695.680224      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751279695.734105      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_19/3917850047.py:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [5]:
# Biến global cho base path
BASE_PATH = "/kaggle/input/KI_minmax_baseline"
# Tuần và số phần fold
weeks = ['week1', 'week2', 'week3', 'week4']
fold_parts = 5

# Tạo five_fold_files
five_fold_files = {
    week: [
        f"{BASE_PATH}/clean_{week}/train/5-folds/data_part_{i}.csv"
        for i in range(1, fold_parts + 1)
    ]
    for week in weeks
}

# Tạo file_validation
file_validation = {
    'week1': [f"{BASE_PATH}/clean_week1/val/val_week1.csv"],
    'week2': [f"{BASE_PATH}/clean_week2/val/val_week1_2.csv"],
    'week3': [f"{BASE_PATH}/clean_week3/val/val_week1_2_3.csv"],
    'week4': [f"{BASE_PATH}/clean_week4/val/val_week1_2_3_4.csv"]
}

# Tạo file_test
file_test = {
    week: [f"{BASE_PATH}/clean_{week}/test/test_{week}.csv"]
    for week in weeks
}

## Tìm siêu tham số tốt nhất cho từng tuần

In [6]:
# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    return focal_loss_fixed

# Tạo hàm train cho từng tuần
def train_week_model(week_number, file_paths_train, file_validataion):
    # Đọc dữ liệu
    train_data = pd.read_csv(file_paths_train)
    val_data = pd.read_csv(file_validataion)
    
    # Tách đặc trưng và nhãn
    X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_train = train_data["classification_encoded"]

    X_val = val_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_val = val_data["classification_encoded"]
    
    # Áp dụng Over-sampling cho dữ liệu huấn luyện bằng SMOTE
    oversampler = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)
    
    # Reshape dữ liệu cho mô hình BiLSTM
    X_train_res = X_train_res.values.reshape(X_train_res.shape[0], X_train_res.shape[1], 1)
    X_val = X_val.values.reshape(X_val.shape[0], X_val.shape[1], 1)
    
    # One-hot encode nhãn
    y_train_res = tf.keras.utils.to_categorical(y_train_res, num_classes=5)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=5)
    
    def build_model(hp):
        inputs = tf.keras.Input(shape=(X_train_res.shape[1], 1))  # Khởi tạo đầu vào
        
        # RNN layer 1
        x = layers.SimpleRNN(
            units=hp.Int('units_1', min_value=32, max_value=256, step=32),
            return_sequences=True
        )(inputs)
        x = layers.Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1))(x)
        
        # RNN layer 2
        x = layers.SimpleRNN(
            units=hp.Int('units_2', min_value=32, max_value=256, step=32),
            return_sequences=False
        )(x)
        x = layers.Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1))(x)
        
        # Lớp đầu ra
        outputs = layers.Dense(5, activation='softmax')(x)
        
        # Khởi tạo mô hình
        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        
        # Compile với Focal Loss
        model.compile(optimizer=tf.keras.optimizers.Adam(
                          learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                      loss=focal_loss(gamma=2., alpha=0.25),
                      metrics=['accuracy'])
        
        return model

    
    # Khởi tạo RandomSearch tuner
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=10,
        executions_per_trial=1,
        directory='my_dir',
        project_name=f'rnn_tuning_week{week_number}'
    )
    
    # Tìm kiếm siêu tham số tốt nhất
    tuner.search(X_train_res, y_train_res,
                 epochs=20,
                 validation_data=(X_val, y_val),
                 batch_size=32)
    
    # Trả về kết quả tối ưu cho tuần
    best_params = tuner.get_best_hyperparameters(num_trials=1)[0]
    return best_params

In [7]:
# Định nghĩa đường dẫn đến dữ liệu cho từng tuần
file_paths_train = {
    week: f"{BASE_PATH}/clean_{week}/train/clean_data_{week}.csv"
    for week in weeks
}

# Định nghĩa file_validation theo quy luật riêng
file_validation = {
    f"week{idx + 1}": f"{BASE_PATH}/clean_week{idx + 1}/val/val_week{'_'.join(str(i) for i in range(1, idx + 2))}.csv"
    for idx in range(len(weeks))
}

In [8]:
# Tìm tham số tốt nhất cho từng tuần
best_params_week1 = train_week_model(1, file_paths_train["week1"], file_validation["week1"])
best_params_week2 = train_week_model(2, file_paths_train["week2"], file_validation["week2"])
best_params_week3 = train_week_model(3, file_paths_train["week3"], file_validation["week3"])
best_params_week4 = train_week_model(4, file_paths_train["week4"], file_validation["week4"])

# In thông tin chi tiết các tham số tối ưu
print("Best Parameters for Week 1:")
for param_name in best_params_week1.values.keys():
    print(f"{param_name}: {best_params_week1.get(param_name)}")

print("\nBest Parameters for Week 2:")
for param_name in best_params_week2.values.keys():
    print(f"{param_name}: {best_params_week2.get(param_name)}")

print("\nBest Parameters for Week 3:")
for param_name in best_params_week3.values.keys():
    print(f"{param_name}: {best_params_week3.get(param_name)}")

print("\nBest Parameters for Week 4:")
for param_name in best_params_week4.values.keys():
    print(f"{param_name}: {best_params_week4.get(param_name)}")


Trial 10 Complete [00h 03m 00s]
val_accuracy: 0.5924344062805176

Best val_accuracy So Far: 0.7858450412750244
Total elapsed time: 00h 29m 56s
Best Parameters for Week 1:
units_1: 224
dropout_1: 0.2
units_2: 256
dropout_2: 0.5
learning_rate: 0.0005105117815909587

Best Parameters for Week 2:
units_1: 256
dropout_1: 0.1
units_2: 192
dropout_2: 0.30000000000000004
learning_rate: 0.0005788602293070543

Best Parameters for Week 3:
units_1: 160
dropout_1: 0.1
units_2: 128
dropout_2: 0.4
learning_rate: 0.000788859060718897

Best Parameters for Week 4:
units_1: 192
dropout_1: 0.1
units_2: 64
dropout_2: 0.1
learning_rate: 0.0004209841475278669


## Danh sách tham số tốt nhất của từng tuần

In [9]:
# Danh sách tham số tốt nhất
best_params = {
    "week1": best_params_week1,
    "week2": best_params_week2,
    "week3": best_params_week3,
    "week4": best_params_week4
}

In [10]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# Xây dựng mô hình BiLSTM
def build_RNN_model(params, input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    
    # RNN layer 1
    x = layers.SimpleRNN(
        units=params.get('units_1'),
        return_sequences=True
    )(inputs)
    x = layers.Dropout(rate=params.get('dropout_1', 0.2))(x)
    
    # RNN layer 2
    x = layers.SimpleRNN(
        units=params.get('units_2', 32),
        return_sequences=False
    )(x)
    x = layers.Dropout(rate=params.get('dropout_2', 0.2))(x)
    
    # Lớp đầu ra
    outputs = layers.Dense(5, activation='softmax')(x)
    
    # Khởi tạo mô hình
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    # Compile với Focal Loss
    model.compile(optimizer=tf.keras.optimizers.Adam(
                      learning_rate=params['learning_rate']),
                  loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
                  metrics=['accuracy'])
    
    return model


In [11]:
# Biến lưu kết quả tổng quát
overall_results_5folds = []

# Lặp qua từng tuần
for week, file_paths in five_fold_files.items():
    print(f"\nProcessing {week} with best parameters...")
    params = best_params[week].values
    print(f"best parameters for {week}: {params}")
    
    # Biến lưu kết quả cho từng tuần
    week_results = {
        "week": week,
        "accuracy_per_fold": [],
        "precision_per_label": [],
        "recall_per_label": [],
        "f1_score_per_label": [],
        "auc_roc_per_label": [],    # AUC từng lớp
        "auc_roc_macro": [],        # AUC macro
        "auc_roc_weighted": [],     # AUC weighted (tự tính)
        "precision_macro": [],
        "recall_macro": [],
        "f1_macro": [],
        "precision_weighted": [],
        "recall_weighted": [],
        "f1_weighted": [],
        "confusion_matrices": [],
        "train_times": [],
        "test_times": []
    }

    # Lặp qua từng fold
    for i in range(len(file_paths)):
        print(f"Fold {i+1}: Using file {file_paths[i]} as test set")
        
        # Tải dữ liệu
        test_data = pd.read_csv(file_paths[i])
        train_data = pd.concat([pd.read_csv(file_paths[j]) for j in range(len(file_paths)) if j != i])
        
        # Tách X và y
        X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                           "course_id", "school", "enroll_time", "classification"])
        y_train = to_categorical(train_data['classification_encoded'], num_classes=5)
        
        X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                         "course_id", "school", "enroll_time", "classification"])
        y_test = to_categorical(test_data['classification_encoded'], num_classes=5)

        # Reshape dữ liệu cho LSTM
        X_train = X_train.to_numpy().reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))
        input_shape = (X_train.shape[1], X_train.shape[2])  # Lấy kích thước từ dữ liệu thực tế
        # Xây dựng mô hình với tham số tốt nhất
        model = build_RNN_model(params, input_shape)
        
        # Bắt đầu tính thời gian huấn luyện
        start_train = time.time()
        model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=32)
        end_train = time.time()
        
        # Bắt đầu tính thời gian kiểm thử
        start_test = time.time()
        y_pred = model.predict(X_test)
        end_test = time.time()
        
        # Tính thời gian và lưu lại
        train_time = end_train - start_train
        test_time = end_test - start_test
        week_results["train_times"].append(train_time)
        week_results["test_times"].append(test_time)

        # Đánh giá mô hình trên tập kiểm thử của fold hiện tại
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        week_results["accuracy_per_fold"].append(accuracy)
        
        # Dự đoán
        y_pred_classes = y_pred.argmax(axis=1)
        y_test_classes = y_test.argmax(axis=1)
        
        # Tính các chỉ số cho mỗi fold
        precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
        conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
        
        # Tính AUC-ROC
        try:
            # Tính AUC macro và theo từng lớp với OvR
            auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
            auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
            # Tính AUC weighted: tính trọng số theo số mẫu của từng lớp
            supports = np.bincount(y_test_classes, minlength=5)
            auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
        except Exception as e:
            print(f"Lỗi khi tính AUC: {e}")
            auc_macro = np.nan
            auc_per_class = [np.nan] * 5
            auc_weighted = np.nan
            
        # Lưu kết quả của fold hiện tại
        week_results["precision_per_label"].append(precision)
        week_results["recall_per_label"].append(recall)
        week_results["f1_score_per_label"].append(f1)
        week_results["auc_roc_per_label"].append(auc_per_class)  # AUC từng lớp
        week_results["auc_roc_macro"].append(auc_macro)          # AUC macro
        week_results["auc_roc_weighted"].append(auc_weighted)      # AUC weighted
        week_results["confusion_matrices"].append(conf_matrix)
        week_results["precision_macro"].append(precision_macro)
        week_results["recall_macro"].append(recall_macro)
        week_results["f1_macro"].append(f1_macro)
        week_results["precision_weighted"].append(precision_weighted)
        week_results["recall_weighted"].append(recall_weighted)
        week_results["f1_weighted"].append(f1_weighted)

    # Tính trung bình cho từng nhãn
    average_precision_per_label = np.mean(week_results["precision_per_label"], axis=0)
    average_recall_per_label = np.nanmean(week_results["recall_per_label"], axis=0)
    average_f1_per_label = np.nanmean(week_results["f1_score_per_label"], axis=0)
    average_auc_per_label = np.nanmean(week_results["auc_roc_per_label"], axis=0)
    average_confusion_matrix = np.nanmean(week_results["confusion_matrices"], axis=0)
    average_train_time = sum(week_results["train_times"]) / len(week_results["train_times"])
    average_test_time = sum(week_results["test_times"]) / len(week_results["test_times"])
    average_accuracy = np.nanmean(week_results["accuracy_per_fold"])
    average_precision_macro = np.nanmean(week_results["precision_macro"])
    average_recall_macro = np.nanmean(week_results["recall_macro"])
    average_f1_macro = np.nanmean(week_results["f1_macro"])
    average_auc_macro = np.nanmean(week_results["auc_roc_macro"])
    average_precision_weighted = np.nanmean(week_results["precision_weighted"])
    average_recall_weighted = np.nanmean(week_results["recall_weighted"])
    average_f1_weighted = np.nanmean(week_results["f1_weighted"])
    average_auc_weighted = np.nanmean(week_results["auc_roc_weighted"])


    # Tạo DataFrame cho precision, recall, f1-score
    labels = np.unique(y_test_classes)  # Lấy nhãn từ y_test_classes
    metrics_df = pd.DataFrame({
        "Label": labels,
        "Average Precision": average_precision_per_label,
        "Average Recall": average_recall_per_label,
        "Average F1-Score": average_f1_per_label,
        "Average AUC": average_auc_per_label
    })
    
    # Tạo DataFrame cho confusion matrix
    confusion_df = pd.DataFrame(average_confusion_matrix, index=labels, columns=labels)
    # In kết quả Accuracy và Macro metrics
    print("\n=== Average Accuracy ===")
    print(f"{average_accuracy:.4f}")
    print("\n=== Average Macro Metrics ===")
    print(f"Macro Precision: {average_precision_macro:.4f}")
    print(f"Macro Recall: {average_recall_macro:.4f}")
    print(f"Macro F1-Score: {average_f1_macro:.4f}")
    print(f"Macro AUC-ROC: {average_auc_macro:.4f}")
    print("\n=== Average Weighted Metrics ===")
    print(f"Weighted Precision: {average_precision_weighted:.4f}")
    print(f"Weighted Recall: {average_recall_weighted:.4f}")
    print(f"Weighted F1-Score: {average_f1_weighted:.4f}")
    print(f"Weighted AUC-ROC: {average_auc_weighted:.4f}")
    print("\n=== Average Metrics per Label ===")
    print(metrics_df)
    print("\n=== Average Confusion Matrix ===")
    print(confusion_df)
    
    # Cập nhật kết quả cho tuần hiện tại
    week_results.update({
        "average_accuracy": average_accuracy,
        "average_precision_macro": average_precision_macro,
        "average_recall_macro": average_recall_macro,
        "average_f1_macro": average_f1_macro,
        "average_auc_macro": average_auc_macro,
        "average_precision_weighted": average_precision_weighted,
        "average_recall_weighted": average_recall_weighted,
        "average_f1_weighted": average_f1_weighted,
        "average_auc_weighted": average_auc_weighted,
        "average_metrics_df": metrics_df,
        "average_confusion_matrix": confusion_df,
        "average_train_times": average_train_time,
        "average_test_times": average_test_time,
    })
    overall_results_5folds.append(week_results)


Processing week1 with best parameters...
best parameters for week1: {'units_1': 224, 'dropout_1': 0.2, 'units_2': 256, 'dropout_2': 0.5, 'learning_rate': 0.0005105117815909587}
Fold 1: Using file /kaggle/input/KI_minmax_baseline/clean_week1/train/5-folds/data_part_1.csv as test set
Epoch 1/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.5600 - loss: 0.2029 - val_accuracy: 0.6885 - val_loss: 0.1187
Epoch 2/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6038 - loss: 0.1591 - val_accuracy: 0.6832 - val_loss: 0.1172
Epoch 3/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6324 - loss: 0.1419 - val_accuracy: 0.6836 - val_loss: 0.1230
Epoch 4/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6491 - loss: 0.1341 - val_accuracy: 0.7000 - val_loss: 0.1092
Epoch 5/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6619 - loss: 0.1251 - val_accuracy: 0.6870 - val_loss: 0.1160
Epoch 6/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6668 - lo

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Fold 2: Using file /kaggle/input/KI_minmax_baseline/clean_week1/train/5-folds/data_part_2.csv as test set
Epoch 1/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.5469 - loss: 0.2168 - val_accuracy: 0.7007 - val_loss: 0.1218
Epoch 2/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6060 - loss: 0.1559 - val_accuracy: 0.6737 - val_loss: 0.1224
Epoch 3/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6299 - loss: 0.1444 - val_accuracy: 0.6820 - val_loss: 0.1217
Epoch 4/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6382 - loss: 0.1354 - val_accuracy: 0.6786 - val_loss: 0.1197
Epoch 5/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6541 - loss: 0.1285 - val_accuracy: 0.6782 - val_loss: 0.1136
Epoch 6/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6600 - loss: 0.1232 - val_accuracy: 0.6950 - val_loss: 0.1162
Epoch 7/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6651 - loss: 0.1190 - val_accuracy: 0.7030 - val_loss: 0.11

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.5589 - loss: 0.2104 - val_accuracy: 0.6701 - val_loss: 0.1439
Epoch 2/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6125 - loss: 0.1573 - val_accuracy: 0.6480 - val_loss: 0.1226
Epoch 3/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6287 - loss: 0.1444 - val_accuracy: 0.6770 - val_loss: 0.1236
Epoch 4/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6365 - loss: 0.1340 - val_accuracy: 0.6808 - val_loss: 0.1277
Epoch 5/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6603 - loss: 0.1273 - val_accuracy: 0.6762 - val_loss: 0.1180
Epoch 6/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6728 - loss: 0.1228 - val_accuracy: 0.6815 - val_loss: 0.1143
Epoch 7/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6691 - loss: 0.1224 - val_accuracy: 0.6827 - val_loss: 0.1122
Epoch 8/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6767 - loss: 0.1185 - val_accuracy: 0.6762 - va

...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Fold 3: Using file /kaggle/input/KI_minmax_baseline/clean_week4/train/5-folds/data_part_3.csv as test set
Epoch 1/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.5668 - loss: 0.1663 - val_accuracy: 0.6785 - val_loss: 0.1189
Epoch 2/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6517 - loss: 0.1279 - val_accuracy: 0.6716 - val_loss: 0.1168
Epoch 3/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6650 - loss: 0.1220 - val_accuracy: 0.6823 - val_loss: 0.1112
Epoch 4/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6793 - loss: 0.1160 - val_accuracy: 0.6995 - val_loss: 0.1075
Epoch 5/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6965 - loss: 0.1144 - val_accuracy: 0.7018 - val_loss: 0.1098
Epoch 6/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7020 - loss: 0.1114 - val_accuracy: 0.7193 - val_loss: 0.1036
Epoch 7/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7007 - loss: 0.1110 - val_accuracy: 0.7212 - val_loss: 0.10

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Fold 5: Using file /kaggle/input/KI_minmax_baseline/clean_week4/train/5-folds/data_part_5.csv as test set
Epoch 1/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.5764 - loss: 0.1801 - val_accuracy: 0.6758 - val_loss: 0.1225
Epoch 2/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6513 - loss: 0.1295 - val_accuracy: 0.6880 - val_loss: 0.1167
Epoch 3/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6633 - loss: 0.1255 - val_accuracy: 0.6808 - val_loss: 0.1123
Epoch 4/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6821 - loss: 0.1174 - val_accuracy: 0.7010 - val_loss: 0.1080
Epoch 5/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6999 - loss: 0.1121 - val_accuracy: 0.7086 - val_loss: 0.1088
Epoch 6/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7010 - loss: 0.1097 - val_accuracy: 0.7128 - val_loss: 0.1037
Epoch 7/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7129 - loss: 0.1053 - val_accuracy: 0.7010 - val_loss: 0.10

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Kết quả cross validation trên 5-folds

In [12]:
# Duyệt qua các tuần trong overall_results
for week_result in overall_results_5folds:
    week = week_result["week"]
    average_train_time = np.mean(week_result["train_times"])
    average_test_time = np.mean(week_result["test_times"])
    average_metrics_df = week_result["average_metrics_df"]
    average_accuracy = np.mean(week_results["accuracy_per_fold"])
    average_confusion_matrix = week_result["average_confusion_matrix"]
    
    # In kết quả
    print(f"\n=== Results for {week} ===")
    print(f"Average Accurancy: {average_accuracy}")
    print(f"Average Train Time: {average_train_time:.4f} seconds")
    print(f"Average Test Time: {average_test_time:.4f} seconds")
    print(f"Average AUC Macro: {average_auc_macro}")
    print(f"Average AUC Weighted: {average_auc_weighted}")
    print("\nAverage Precision, Recall, F1-Score, AUC-ROC per Label:")
    print(average_metrics_df)
    print("\nAverage Confusion Matrix:")
    print(average_confusion_matrix)



=== Results for week1 ===
Average Accurancy: 0.7577773809432984
Average Train Time: 25.7155 seconds
Average Test Time: 1.1152 seconds
Average AUC Macro: 0.8705419071494553
Average AUC Weighted: 0.8858597803206155

Average Precision, Recall, F1-Score, AUC-ROC per Label:
   Label  Average Precision  Average Recall  Average F1-Score  Average AUC
0      0           0.644598        0.469333          0.533092     0.840204
1      1           0.000000        0.000000          0.000000     0.806894
2      2           0.637935        0.444117          0.515598     0.848647
3      3           0.565874        0.233533          0.312425     0.866025
4      4           0.750327        0.926786          0.828507     0.874481

Average Confusion Matrix:
       0    1     2     3       4
0  281.6  0.0  15.2  13.6   289.6
1   25.2  0.0   0.6   4.8    57.0
2   25.8  0.0  73.0   5.2    60.4
3   34.0  0.0   2.6  39.0    91.4
4   81.0  0.0  25.6  10.8  1486.0

=== Results for week2 ===
Average Accurancy: 0.

## Kiểm tra trên tập test

In [13]:
# Mảng lưu dữ liệu của các tuần
results = []

def process_week(week_num, best_params, results):
    print(f"\n=== Processing Week {week_num} ===")
    params = best_params[f"week{week_num}"].values
    # Đường dẫn tới dữ liệu tuần tương ứng
    train_path = f"{BASE_PATH}/clean_week{week_num}/train/clean_data_week{week_num}.csv"
    test_path = f"{BASE_PATH}/clean_week{week_num}/test/test_week{week_num}.csv"
    
    # Load dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Tách X và y
    X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                       "course_id", "school", "enroll_time", "classification"])
    y_train = train_data['classification_encoded']
    
    X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                     "course_id", "school", "enroll_time", "classification"])
    y_test = test_data['classification_encoded']

    # Áp dụng SMOTE cho tập huấn luyện
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Chuyển đổi nhãn sang dạng one-hot
    y_train_resampled = to_categorical(y_train_resampled, num_classes=5)
    y_test = to_categorical(y_test, num_classes=5)

    # Reshape dữ liệu cho LSTM
    X_train_resampled = X_train_resampled.to_numpy().reshape((X_train_resampled.shape[0], 1, X_train_resampled.shape[1]))
    X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))
    
    # Xây dựng mô hình với tham số tốt nhất
    input_shape = (X_train_resampled.shape[1], X_train_resampled.shape[2])  # Lấy kích thước từ dữ liệu thực tế
    model = build_RNN_model(params, input_shape)
    
    # Huấn luyện mô hình
    start_train = time.time()
    model.fit(X_train_resampled, y_train_resampled, epochs=20, validation_split=0.1, batch_size=32)
    end_train = time.time()
    
    # Kiểm thử mô hình
    start_test = time.time()
    y_pred = model.predict(X_test)
    end_test = time.time()
    
    # Tính thời gian huấn luyện và kiểm thử
    train_time = end_train - start_train
    test_time = end_test - start_test
    
    # Đánh giá mô hình
    y_pred_classes = y_pred.argmax(axis=1)
    y_test_classes = y_test.argmax(axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    
    # Tính AUC-ROC (với one-vs-rest)
    try:
        auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
        auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
        # Tính AUC weighted tự tính theo trọng số mẫu của từng lớp
        supports = np.bincount(y_test_classes, minlength=5)
        auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
    except Exception as e:
        print(f"Lỗi khi tính AUC: {e}")
        auc_macro = np.nan
        auc_per_class = [np.nan] * 5
        auc_weighted = np.nan

    # Lưu kết quả vào mảng
    results.append({
        "week": week_num,
        "train_time": train_time,
        "test_time": test_time,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "accuracy": accuracy,
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "precision_weighted": precision_weighted,
        "recall_weighted": recall_weighted,
        "f1_weighted": f1_weighted,
        "auc_macro": auc_macro,
        "auc_weighted": auc_weighted,
        "auc_per_class": auc_per_class,
        "confusion_matrix": conf_matrix
    })
    
    # In kết quả chi tiết
    print("\n=== Precision, Recall, F1-Score per Label ===")
    print(pd.DataFrame({
        "Label": np.unique(y_test_classes),
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }))

    print("\n=== Macro Averages & Accuracy ===")
    print(f"Macro Precision: {precision_macro:.4f}")
    print(f"Macro Recall: {recall_macro:.4f}")
    print(f"Macro F1-Score: {f1_macro:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    
    print("\n=== Weighted Averages ===")
    print(f"Weighted Precision: {precision_weighted:.4f}")
    print(f"Weighted Recall: {recall_weighted:.4f}")
    print(f"Weighted F1-Score: {f1_weighted:.4f}")
    
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {auc_macro:.4f}")
    print(f"AUC Weighted: {auc_weighted:.4f}")
    print(f"AUC per Label: {auc_per_class}")
    
    print("\n=== Confusion Matrix ===")
    print(pd.DataFrame(conf_matrix, index=np.unique(y_test_classes), columns=np.unique(y_test_classes)))
    
    print(f"\nTrain Time: {train_time:.2f} seconds")
    print(f"Test Time: {test_time:.2f} seconds")

In [14]:
process_week(1, best_params, results)


=== Processing Week 1 ===
Epoch 1/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.3870 - loss: 0.2727 - val_accuracy: 0.4280 - val_loss: 0.3233
Epoch 2/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4957 - loss: 0.1976 - val_accuracy: 0.5463 - val_loss: 0.2406
Epoch 3/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5342 - loss: 0.1840 - val_accuracy: 0.4450 - val_loss: 0.2484
Epoch 4/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5400 - loss: 0.1784 - val_accuracy: 0.5470 - val_loss: 0.2252
Epoch 5/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5558 - loss: 0.1727 - val_accuracy: 0.4832 - val_loss: 0.2647
Epoch 6/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5612 - loss: 0.1703 - val_accuracy: 0.4869 - val_loss: 0.2782
Epoch 7/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5690 - loss: 0.1685 - val_accuracy: 0.6071 - val_loss: 0.2366
Epoch 8/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accura

In [15]:
process_week(2, best_params, results)


=== Processing Week 2 ===
Epoch 1/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.4185 - loss: 0.2371 - val_accuracy: 0.4240 - val_loss: 0.2948
Epoch 2/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5318 - loss: 0.1841 - val_accuracy: 0.5435 - val_loss: 0.2500
Epoch 3/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5654 - loss: 0.1716 - val_accuracy: 0.5203 - val_loss: 0.2520
Epoch 4/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5818 - loss: 0.1640 - val_accuracy: 0.5740 - val_loss: 0.2433
Epoch 5/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5934 - loss: 0.1597 - val_accuracy: 0.5992 - val_loss: 0.2370
Epoch 6/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5982 - loss: 0.1576 - val_accuracy: 0.6615 - val_loss: 0.2025
Epoch 7/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5968 - loss: 0.1562 - val_accuracy: 0.6318 - val_loss: 0.2495
Epoch 8/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accura

In [16]:
process_week(3, best_params, results)


=== Processing Week 3 ===
Epoch 1/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.4053 - loss: 0.2424 - val_accuracy: 0.4413 - val_loss: 0.3194
Epoch 2/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5292 - loss: 0.1840 - val_accuracy: 0.4181 - val_loss: 0.2729
Epoch 3/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5578 - loss: 0.1735 - val_accuracy: 0.6059 - val_loss: 0.2639
Epoch 4/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5760 - loss: 0.1658 - val_accuracy: 0.3520 - val_loss: 0.3596
Epoch 5/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5840 - loss: 0.1638 - val_accuracy: 0.5587 - val_loss: 0.2496
Epoch 6/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5856 - loss: 0.1631 - val_accuracy: 0.7069 - val_loss: 0.1603
Epoch 7/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5951 - loss: 0.1596 - val_accuracy: 0.6575 - val_loss: 0.2096
Epoch 8/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accurac

In [17]:
process_week(4, best_params, results)


=== Processing Week 4 ===
Epoch 1/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.4395 - loss: 0.2160 - val_accuracy: 0.4206 - val_loss: 0.2624
Epoch 2/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5554 - loss: 0.1740 - val_accuracy: 0.5550 - val_loss: 0.2419
Epoch 3/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5849 - loss: 0.1633 - val_accuracy: 0.5804 - val_loss: 0.2517
Epoch 4/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5964 - loss: 0.1556 - val_accuracy: 0.6914 - val_loss: 0.1995
Epoch 5/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6112 - loss: 0.1517 - val_accuracy: 0.6580 - val_loss: 0.2009
Epoch 6/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6186 - loss: 0.1485 - val_accuracy: 0.6747 - val_loss: 0.2118
Epoch 7/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6207 - loss: 0.1462 - val_accuracy: 0.5809 - val_loss: 0.2610
Epoch 8/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accura

In [18]:
# Hiển thị dữ liệu của các tuần
print("\n=== Summary Results for All Weeks ===")
for result in results:
    print(f"Week {result['week']}:")
    print(f"  Train Time: {result['train_time']:.2f} seconds")
    print(f"  Test Time: {result['test_time']:.2f} seconds")
    print(f"  Accurancy: {result['accuracy']}")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1-Score: {result['f1_score']}")
    print(f"  Macro Precision: {result['precision_macro']}")
    print(f"  Macro Recall: {result['recall_macro']}")
    print(f"  Macro F1-Score: {result['f1_macro']}")
    print(f"  Confusion Matrix:\n{result['confusion_matrix']}")
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {result['auc_macro']:.4f}")
    print(f"AUC Weighted: {result['auc_weighted']:.4f}")
    print(f"AUC per Label: {result['auc_per_class']}")


=== Summary Results for All Weeks ===
Week 1:
  Train Time: 63.28 seconds
  Test Time: 1.15 seconds
  Accurancy: 0.7158536585365853
  Precision: [0.71299094 0.16589862 0.55102041 0.50387597 0.90520231]
  Recall: [0.62933333 0.66666667 0.52427184 0.61904762 0.78065803]
  F1-Score: [0.66855524 0.26568266 0.53731343 0.55555556 0.83832976]
  Macro Precision: 0.5677976486723307
  Macro Recall: 0.6439954979260092
  Macro F1-Score: 0.5730873300930215
  Confusion Matrix:
[[236  71   9   3  56]
 [ 10  36   0   1   7]
 [ 19  21  54   2   7]
 [  9  13   6  65  12]
 [ 57  76  29  58 783]]

=== AUC-ROC ===
AUC Macro: 0.8850
AUC Weighted: 0.8994
AUC per Label: [0.90514993 0.87661132 0.85364567 0.8850287  0.90461426]
Week 2:
  Train Time: 61.28 seconds
  Test Time: 1.13 seconds
  Accurancy: 0.7036585365853658
  Precision: [0.76865672 0.1634981  0.46564885 0.53278689 0.91004673]
  Recall: [0.54933333 0.7962963  0.59223301 0.61904762 0.77666999]
  F1-Score: [0.6407465  0.27129338 0.52136752 0.57268722